At the moment, the script works with only one card set at a time.  Set the card set UUID:

In [106]:
#SET_UUID = '8a3273ca-1ccd-4e07-913b-766fcc49fe6f'
SET_UUID = '17b90ca1-7e3e-4891-99da-954a7833e88e'

Set whether you want to generate all cards from scratch (`True`) or to update only the cards, changed since the previous script run (`False`):

In [107]:
FROM_SCRATCH = True

Run the cells below:

In [108]:
import hashlib
import os
import re
import requests
import shutil
import xlwings as xw
import xml.etree.ElementTree as ET
import yaml
import zipfile

PROJECT_FOLDER = 'Frogmorton'
SHEET_NAME = 'setExcel'

CONFIGURATION_PATH = 'configuration.yaml'
IMAGES_PATH = os.path.join(PROJECT_FOLDER, 'imagesRaw')
IMAGES_ZIP_PATH = '{}/Export/'.format(os.path.split(PROJECT_FOLDER)[-1])
MACROS_PATH = 'macros.xlsm'
MACROS_COPY_PATH = 'macros_copy.xlsm'
OCTGN_ZIP_PATH = 'imagesOCTGN/a21af4e8-be4b-4cda-a6b6-534f9717391f/Sets'
OUTPUT_DB_PATH = os.path.join('Output', 'DB')
OUTPUT_IMAGES_PATH = 'imagesEons'
OUTPUT_OCTGN_PATH = os.path.join('Output', 'OCTGN')
PROJECT_PATH = 'setGenerator.seproject'
SET_EONS_PATH = 'setEons'
XML_PATH = os.path.join(PROJECT_FOLDER, 'XML')

In [109]:
# Read configuration
with open(CONFIGURATION_PATH, 'r') as f_conf:
    conf = yaml.safe_load(f_conf)

In [110]:
# Download the cards spreadsheet from Google Drive
sheet_path = '{}.{}'.format(SHEET_NAME, conf['sheet_type'])
if conf['sheet_type'] == 'xlsm':
    url = 'https://drive.google.com/uc?export=download&id={}'.format(conf['sheet_gdid'])
else:
    url = 'https://docs.google.com/spreadsheets/d/{}/export?format=xlsx'.format(conf['sheet_gdid'])

with open(sheet_path, 'wb') as f_sheet:
    f_sheet.write(requests.get(url).content)

In [111]:
# Backup the previous setEons.xml
if os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml')):
    _ = shutil.move(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'),
                os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))

if FROM_SCRATCH and os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old')):
    os.remove(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))

In [112]:
# Copy Excel Sheet over to sheet that contains the macros, and run the macros.
# This will create two .xml files. One in the setEons folder called setEons.xml,
# which contains the card data needed by Strange Eons.
# The second will be placed in the Output/OCTGN folder, which should be copied over to your OCTGN folder.

CARD_DATA_RANGE = 'A2:AU1001'

_ = shutil.copyfile(MACROS_PATH, MACROS_COPY_PATH)

set_name = None

app = xw.App(visible=False)  # if you want Excel to run in background
xlws = {}

xlwb1 = xw.Book(sheet_path)
try:
    xlws['ws1a'] = xlwb1.sheets['Sets']
    xlws['ws1b'] = xlwb1.sheets['Card Data']

    set_row = None
    for row in range(3, 102):
        if xlws['ws1a'].range((row, 1)).value == SET_UUID:
            set_row = row
            set_name = xlws['ws1a'].range((row, 2)).value
            break

    if set_row:
        
        xlwb2 = xw.Book(MACROS_COPY_PATH)
        try:
            xlws['ws2a'] = xlwb2.sheets['Sets']
            xlws['ws2b'] = xlwb2.sheets['Card Data']

            newVal = xlws['ws1a'].range('A{}:B{}'.format(set_row, set_row)).value  # get set id/name
            xlws['ws2a'].range('A3:B3').value = newVal  # update set id/name

            newVal = xlws['ws1b'].range(CARD_DATA_RANGE).value  # get card data
            xlws['ws2b'].range(CARD_DATA_RANGE).value = newVal  # update card data

            xlws['ws2b'].range(CARD_DATA_RANGE).api.Sort(
                Key1=xlws['ws2b'].range('Set').api,
                Order1=xw.constants.SortOrder.xlAscending,
                Key2=xlws['ws2b'].range('CardNumber').api,
                Order2=xw.constants.SortOrder.xlAscending)

            # This macro will generate a folder under Output/OCTGN, which should be copied over
            # to OCTGN/GameDatabase/a21af4e8-be4b-4cda-a6b6-534f9717391f/Sets 
            macro1 = xlwb2.macro('SaveOCTGN')
            macro1()

            # This macro will generate a file called setEons.xml, which is used by the Strange Eons script.
            macro2 = xlwb2.macro('SaveXML')
            macro2()

            xlwb2.save()
        finally:
            xlwb2.close()
    else:
        print('Set {} not found in the spreadsheet'.format(SET_UUID))    
finally:
    xlwb1.close()

In [113]:
# Find properties with a given name
def find_properties(parent, name):
    return [p for p in parent if p.attrib.get('name') == name]

# Get new or existing property with a given name
def get_property(parent, name):
    properties = find_properties(parent, name)
    if properties:
        prop = properties[0]
    else:
        prop = ET.SubElement(parent, 'property')
        prop.set('name', name)

    return prop

# Delete old image files
for _, _, filenames in os.walk(IMAGES_PATH):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            os.remove(os.path.join(IMAGES_PATH, filename))

    break

# Parse artwork images
images = {}
for _, _, filenames in os.walk(conf['artwork_path']):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            card_id_side = '_'.join(filename.split('_')[:2])
            images[card_id_side] = filename

    break

# Update setEons.xml with artwork, artists and encounter set numbers
tree = ET.parse(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'))
root = tree.getroot()
encounter_sets = {}
encounter_cards = {}

for card in root[0]:
    card_type = find_properties(card, 'Type')[0].attrib['value']
    encounter_set = find_properties(card, 'Encounter Set')
    if card_type != 'Quest' and encounter_set:
        encounter_set = encounter_set[0].attrib['value']
        encounter_cards[card.attrib['id']] = encounter_set
        prop = get_property(card, 'Encounter Set Number')
        prop.set('value', str(encounter_sets.get(encounter_set, 0) + 1))
        quantity = int(find_properties(card, 'Quantity')[0].attrib['value'])
        encounter_sets[encounter_set] = encounter_sets.get(encounter_set, 0) + quantity

    filename = images.get('{}_{}'.format(card.attrib['id'], 'A'))
    if filename:
        prop = get_property(card, 'Artwork')
        prop.set('value', filename)
        prop = get_property(card, 'Artwork Size')
        prop.set('value', str(os.path.getsize(os.path.join(conf['artwork_path'], filename))))
        prop = get_property(card, 'Artwork Modified')
        prop.set('value', str(int(os.path.getmtime(os.path.join(conf['artwork_path'], filename)))))

        artist = find_properties(card, 'Artist')
        if not artist and '_Artist_' in filename:
            artist_value = '.'.join('_Artist_'.join(
                filename.split('_Artist_')[1:]).split('.')[:-1]).replace('_', ' ')
            prop = get_property(card, 'Artist')
            prop.set('value', artist_value)

    filename = images.get('{}_{}'.format(card.attrib['id'], 'B'))
    alternate = [a for a in card if a.attrib.get('type') == 'B']
    if filename and alternate:
        alternate = alternate[0]
        prop = get_property(alternate, 'Artwork')
        prop.set('value', filename)
        prop = get_property(alternate, 'Artwork Size')
        prop.set('value', str(os.path.getsize(os.path.join(conf['artwork_path'], filename))))
        prop = get_property(alternate, 'Artwork Modified')
        prop.set('value', str(int(os.path.getmtime(os.path.join(conf['artwork_path'], filename)))))

        artist = find_properties(alternate, 'Artist')
        if not artist and '_Artist_' in filename:
            artist_value = '.'.join('_Artist_'.join(
                filename.split('_Artist_')[1:]).split('.')[:-1]).replace('_', ' ')
            prop = get_property(alternate, 'Artist')
            prop.set('value', artist_value)

# Update setEons.xml with encounter set total and card hash
for card in root[0]:
    if card.attrib['id'] in encounter_cards:
        prop = get_property(card, 'Encounter Set Total')
        prop.set('value', str(encounter_sets[encounter_cards[card.attrib['id']]]))

    card_hash = hashlib.md5(re.sub('\n\s*', '', ET.tostring(card, encoding='unicode').strip()).encode()).hexdigest()
    card.set('hash', card_hash)

# Mark cards, which were not changed since the previous script run and copy artwork images into the project
old_hashes = {}
skip_ids = set()
if os.path.exists(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old')):
    tree_old = ET.parse(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml.old'))
    root_old = tree_old.getroot()
    for card in root_old[0]:
        old_hashes[card.attrib['id']] = card.attrib['hash']

for card in root[0]:
    if old_hashes.get(card.attrib['id']) == card.attrib['hash']:
        skip_ids.add(card.attrib['id'])
        card.set('skip', '1')
    else:
        filename = find_properties(card, 'Artwork')
        if filename:
            filename = filename[0].attrib['value']
            _ = shutil.copyfile(os.path.join(conf['artwork_path'], filename),
                                os.path.join(IMAGES_PATH, filename))

        alternate = [a for a in card if a.attrib.get('type') == 'B']
        if alternate:
            alternate = alternate[0]
            filename = find_properties(alternate, 'Artwork')
            if filename:
                filename = filename[0].attrib['value']
                _ = shutil.copyfile(os.path.join(conf['artwork_path'], filename),
                                    os.path.join(IMAGES_PATH, filename))

tree.write(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'))

# Copy setEons.xml into the project
_ = shutil.copyfile(os.path.join(SET_EONS_PATH, SET_UUID, 'setEons.xml'),
                    os.path.join(XML_PATH, 'setEons.xml'))

In [114]:
# Create Strange Eons project archive
with zipfile.ZipFile(PROJECT_PATH, 'w') as zip_obj:
    for root, _, filenames in os.walk(PROJECT_FOLDER):
        for filename in filenames:
            zip_obj.write(os.path.join(root, filename))

### Manual Strange Eons Part:

1. Open `setGenerator.seproject` generated above.
2. Run `Script/makeCards` script by double clicking it.  This will take the templates, fill them with values from `setEons.xml` and export results in all the required formats.
3. Once completed, close Strange Eons.

Run the cells below:

In [ ]:
for folder in ('jpg300NoBleed', 'png300Bleed', 'png300NoBleed', 'png800Bleed'):
    # Create image folders if they don't exist yet
    if not os.path.exists(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID)):
        os.mkdir(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID))

    # Delete old image files
    for _, _, filenames in os.walk(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID)):
        for filename in filenames:
            if filename.split('.')[-1] in ('jpg', 'png'):
                card_id = filename[50:86]
                if card_id not in skip_ids:
                    os.remove(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID, filename))

        break

    # Copy generated images from the project zip
    with zipfile.ZipFile(PROJECT_PATH) as zip_obj:
        filelist = [f for f in zip_obj.namelist()
                    if f.startswith('{}{}'.format(IMAGES_ZIP_PATH, folder))
                    and f.split('.')[-1] in ('jpg', 'png')]
        for filename in filelist:
            output_filename = os.path.split(filename)[-1]
            output_filename = re.sub('-2-1(?=\.(?:jpg|png)$)', '-2', output_filename)
            parts = output_filename.split('-')
            if parts[1] == 'p':
                for i in range(1, 4):
                    parts[1] = str(i)
                    output_filename = '-'.join(parts)
                    with zip_obj.open(filename) as zip_file:
                        with open(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID,
                                  output_filename), 'wb') as output_file:
                            _ = shutil.copyfileobj(zip_file, output_file)
            else:
                with zip_obj.open(filename) as zip_file:
                    with open(os.path.join(OUTPUT_IMAGES_PATH, folder, SET_UUID,
                              output_filename), 'wb') as output_file:
                        _ = shutil.copyfileobj(zip_file, output_file)

In [105]:
# Create an OCTGN folder if it doesn't exist yet
if not os.path.exists(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)):
    os.mkdir(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID))

# Create a DB folder if it doesn't exist yet
if not os.path.exists(os.path.join(OUTPUT_DB_PATH, set_name)):
    os.mkdir(os.path.join(OUTPUT_DB_PATH, set_name))

# Delete old image files in DB folder
for _, _, filenames in os.walk(os.path.join(OUTPUT_DB_PATH, set_name)):
    for filename in filenames:
        if filename.split('.')[-1] in ('jpg', 'png'):
            os.remove(os.path.join(OUTPUT_DB_PATH, set_name, filename))

    break

# Make an OCTGN image pack and copy to DB folder
pack_path = os.path.join(OUTPUT_OCTGN_PATH, SET_UUID, 'imagePack_{}.o8c'.format(SET_UUID))
known_filenames = set()
with zipfile.ZipFile(pack_path, 'w', zipfile.ZIP_DEFLATED) as zip_obj:
    for _, _, filenames in os.walk(os.path.join(OUTPUT_IMAGES_PATH, 'jpg300NoBleed', SET_UUID)):
        for filename in filenames:
            if filename.split('.')[-1] not in ('jpg', 'png'):
                continue

            filepath = os.path.join(OUTPUT_IMAGES_PATH, 'jpg300NoBleed', SET_UUID, filename)
            parts = filename.split('.')
            parts[0] = re.sub('-1$', '', re.sub('-2$', '.B', parts[0]))
            octgn_filename = '.'.join(parts)
            if len(octgn_filename) > 50: 
                octgn_filename = octgn_filename[50:]

            if octgn_filename not in known_filenames:
                known_filenames.add(octgn_filename)
                newfile = '{}/{}/Cards/{}'.format(OCTGN_ZIP_PATH, SET_UUID, octgn_filename)
                zip_obj.write(filepath, newfile)
                
            db_filename = '{}-{}{}{}'.format(filename[:3], re.sub('-+$', '',filename[8:50]),
                                           re.sub('-1$', '', filename[86:88]), filename[88:])
            _ = shutil.copyfile(filepath, os.path.join(OUTPUT_DB_PATH, set_name, db_filename))

        break

Now there should be `set.xml` and `imagePack_<octgnid>.o8c` files in `Output/OCTGN/<octgnid>` folder.  Open the `imagePack` file using the "Add Image Packs" button from within OCTGN to add it.

In [ ]:
# Copy set folder to OCTGN directory
#from distutils.dir_util import copy_tree

#dirOCTGN = os.path.expanduser('~') + '\\Documents\\OCTGN\\'
#if not os.path.isdir(dirOCTGN):
#    print('Invalid path!')

#setspath = dirOCTGN + '\\GameDatabase\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\'

#if os.path.isdir(dirOCTGN):
#    if os.path.isdir(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID)):
#        copy_tree(os.path.join(OUTPUT_OCTGN_PATH, SET_UUID), os.path.join(setspath, SET_UUID))
#        print('Copied successfully.')
#    else:
#        print('No folder found to copy.')
#else:
#    print('OCTGN Sets folder not found.')